In [2]:
import pandas as pd

In [3]:
# getting the data
df = pd.read_csv("lending_train.csv")
df = df.drop(columns=['ID', 'race'])
df

,requested_amnt,loan_duration,employment,employment_length,reason_for_loan,extended_reason,annual_income,debt_to_income_ratio,employment_verified,public_bankruptcies,...,delinquency_last_2yrs,fico_score_range_low,fico_score_range_high,fico_inquired_last_6mths,months_since_last_delinq,revolving_balance,total_revolving_limit,type_of_application,any_tax_liens,loan_paid
0,32000.0,60 months,SVP,4 years,debt_consolidation,Debt consolidation,250000.0,16.35,Verified,0.0,...,0.0,775.0,779.0,0.0,NaN,22480.0,105700.0,Individual,0.0,1
1,6000.0,36 months,METAL FABRICATION SUPERVISOR,10+ years,home_improvement,Home improvement,70000.0,15.22,Source Verified,0.0,...,0.0,650.0,654.0,1.0,30.0,6313.0,14600.0,Individual,0.0,1
2,4200.0,36 months,insurance collector specialist,10+ years,debt_consolidation,Debt consolidation,37000.0,20.99,Not Verified,0.0,...,1.0,665.0,669.0,0.0,11.0,4482.0,10300.0,Individual,0.0,1
3,9725.0,36 months,Kitchen Manager,10+ years,debt_consolidation,Debt consolidation,45000.0,37.76,Source Verified,0.0,...,0.0,705.0,709.0,0.0,NaN,21705.0,25200.0,Individual,0.0,1
4,24000.0,36 months,Appeals Specialist,10+ years,debt_consolidation,Debt consolidation,86000.0,32.10,Not Verified,0.0,...,0.0,650.0,654.0,0.0,76.0,30406.0,104200.0,Individual,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,8000.0,36 months,Human Resources Analyst,2 years,home_improvement,Home improvement,71000.0,25.93,Verified,1.0,...,1.0,665.0,669.0,2.0,17.0,14913.0,30800.0,Individual,0.0,1
999996,4000.0,36 months,Support Manager,6 years,debt_consolidation,Debt consolidation,38000.0,13.33,Verified,0.0,...,0.0,685.0,689.0,0.0,77.0,4813.0,8900.0,Individual,0.0,1
999997,11000.0,60 months,software specialist,< 1 year,debt_consolidation,Debt consolidation,57500.0,9.41,Verified,0.0,...,0.0,725.0,729.0,1.0,NaN,10834.0,13000.0,Individual,0.0,1
999998,14400.0,36 months,QMS,1 year,credit_card,Credit card refinancing,48000.0,16.50,Not Verified,0.0,...,0.0,690.0,694.0,1.0,NaN,15745.0,20600.0,Individual,0.0,1


In [4]:
# normalizing columns
columns_to_norm = ['requested_amnt', 'annual_income', 'debt_to_income_ratio', 'fico_score_range_low', 'fico_score_range_high', 'revolving_balance', 'total_revolving_limit']
for col in columns_to_norm:
    df[col] = (df[col] - df[col].mean()) / df[col].std()

In [ ]:
# one-hot encoding necessary columns
cols_to_one_hot_encode = ['type_of_application', 'reason_for_loan', 'extended_reason', 'employment_verified']
new_cols = pd.get_dummies(df, columns=cols_to_one_hot_encode)
df = df.join(new_cols)
df = df.drop(columns=cols_to_one_hot_encode)

In [ ]:
df